In [ ]:
from __future__ import division 
import numpy as np
import glob
import sys
import pdb
import matplotlib.pyplot as plt # plt ??????
import matplotlib.image as mpimg # mpimg ??????
import os
import csv
from matplotlib import gridspec 
import os
import sys
import mmap
import caffe
from tqdm import *
def search(path, word):
    l1=[]
    for filename in os.listdir(path):
        fp = os.path.join(path, filename)
        if os.path.isfile(fp) and word in filename:
            l1.append(fp)
        elif os.path.isdir(fp):
            l1.extend(search(fp, word))
    return l1    

In [ ]:
def vote(filename):
    caffe.set_mode_gpu()
    caffe.set_device(0)
    model_def = 'deploy_SE_bn4.prototxt'
    numm=100
    mode=search("./", "_"+filename.split('/')[-2]+"_")
    for modelname in mode:
        print (model_def,"loading............")
        print (modelname,"loading............")
        model_weights = modelname        
        mu = np.load('train_224_mean.npy')
        mu = mu.mean(1).mean(1)  # average over pixels to obtain the mean (BGR) pixel values
        net= caffe.Net(model_def,      # defines the structure of the model
                       model_weights ,  # contains the trained weights
                       caffe.TEST)     # use test mode (e.g., don't perform dropout)

        transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})

        transformer.set_transpose('data', (2,0,1))  # move image channels to outermost dimension
        transformer.set_mean('data', mu)            # subtract the dataset-mean value in each channel
        transformer.set_raw_scale('data', 255)      # rescale from [0, 1] to [0, 255]
        transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR

        net.blobs['data'].reshape(numm,        # batch size
                                  3,         # 3-channel (BGR) images
                                  224, 224)  # image size is 227x227

        #输出结果暂存地
        out=np.zeros((0,2))            
        ii=0
        allfile=open(filename,'r')
        done = 0
        while not done:
                line = allfile.readline()
                if(line != ''): 
                    realname=line[0:-1].split(' ')[0]
                    image = caffe.io.load_image(realname)
                    transformed_image = transformer.preprocess('data', image)
                    # copy the image data into the memory allocated for the net
                    net.blobs['data'].data[ii] = transformed_image
                    ii=ii+1
                    if ii>numm-1:
                        output = net.forward()
                        out=np.vstack((out,output['prob']))
                        ii=0                    
                else:
                    output = net.forward()
                    out=np.vstack((out,output['prob']))
                    done = 1
        allfile.close()  
        net=None
        t=open("result/"+modelname.split("/")[-1][0]+"_"+filename.split('/')[-2]+"_"+filename.split('/')[-1].split('.')[0]+".txt","w")
        np.savetxt(t,out)

In [ ]:
#extract patch model result 
#modify the txtfile to global/ will extract global model result . Don't forget to modify the result dir
if __name__=='__main__':
    txtfile="patch/"
    filen=search(txtfile,'test.txt')
    for f in filen:
        print (f,"loading............")
        vote(f) 

In [ ]:
#compute various matrix on image level
def anamean(Global,Patch,groundtruth,L):
    global_name=Global
    fileground=open(groundtruth,'r')
    Global=np.loadtxt(Global)    
    Patch=np.loadtxt(Patch)
    corr0=0
    corr1=0
    label0=0
    label1=0
    wrong=0    
    re_list=[]
    for (num,picturename) in enumerate(fileground):        
        realname=picturename.split(' ')[0]
        reallabel=int(picturename[0:-1].split(' ')[-1])
        
        patchvote=Patch[num*15:(num+1)*15]
        delline=[]
        #若只计算patch不投票的情况 从这里到下面开始注释
        for num2,x in enumerate(patchvote):
            if max(x)-min(x)<0.3:
                delline.append(num2)
        patchvote=np.delete(patchvote,delline,axis=0)
        if reallabel:
            label1+=1
        else:
            label0+=1
        siz,yy=patchvote.shape
        patchvote=np.sum(patchvote, axis=0)
        patchvote=patchvote/siz
        globalvote=Global[num:num+1]     
        Glocan=1.0
        pre=L[0]*patchvote+Glocan*globalvote
        prelabel=pre.argmax()
        if prelabel==reallabel==0:
            corr0=corr0+1
        elif prelabel==reallabel==1:
            corr1=corr1+1
        else:
            wrong=wrong+1    
        re_list.append(pre)
#计算只有patch
#         for xx in patchvote:
#             if reallabel:
#                 label1+=1
#             else:
#                 label0+=1
#             prelabel=xx.argmax()
#             if prelabel==reallabel==0:
#                 corr0=corr0+1
#             elif prelabel==reallabel==1:
#                 corr1=corr1+1
#             else:
#                 wrong=wrong+1   

#     res=open('result/result11.txt','a+')
#     res.write(global_name.split('/')[-1].split('_')[0]+" "+groundtruth.split('/')[-2]+" correct0:"+str(corr0)+" correct1:"+str(corr1)+" label0:"+str(label0)+" label1:"+str(label1)+" pro:"+str(float(corr0+corr1)/float(corr0+corr1+wrong))+'\n\n')
#     res.close()
    
    #五折一起投票的话 返回上面那个数字
    return re_list
#     return float(corr0+corr1)/float(corr0+corr1+wrong)
#计算图片级别
def fivevote(list_pro):
    corr0=0
    corr1=0
    label0=0
    label1=0
    wrong=0    
    groud="global/test.txt"
    fileground=open(groud,'r')
    for (num,picturename) in enumerate(fileground):
        reallabel=int(picturename[0:-1].split(' ')[-1])
        if reallabel:
            label1+=1
        else:
            label0+=1
        #using sum
#         pro_l=[0,0]
#         for ii in range(5):
#             pro_l+=list_pro[ii][num]
#         prelabel=pro_l[0].argmax()
         #using max
        max_value=-1
        prelabel=-1
        for ii in range(5):
            for ii_2 in range(2):
                if list_pro[ii][num][0][ii_2]>max_value:
                    max_value=list_pro[ii][num][0][ii_2]
                    prelabel=ii_2    
        
        if prelabel==reallabel==0:
            corr0=corr0+1
        elif prelabel==reallabel==1:
            corr1=corr1+1
        else:
            wrong=wrong+1    
    res=open('result/result11.txt','a+')
    res.write(groud.split('/')[-1].split('_')[0]+" "+groud.split('/')[-2]+" correct0:"+str(corr0)+" correct1:"+str(corr1)+" label0:"+str(label0)+" label1:"+str(label1)+" pro:"+str(float(corr0+corr1)/float(corr0+corr1+wrong))+'\n\n')
    res.close()
    return float(corr0+corr1)/float(corr0+corr1+wrong)
if __name__=='__main__':
    if 'result11.txt' in os.listdir('result/'):
        os.remove('result/result11.txt')
    txtfile2="result/"
    filelist=search(txtfile2,'_patch')
    dic1={}
    dic1['40X']=[]
    L=[1.9,5.9,3.8,3.65]
#     L=[0,0,0,0]
    for fpatch in filelist:
                gro=fpatch.split('/')[-1].split('.')[0]
                groud="/home/devin/Pictures/ICIAR2018_part2crop/tools/my_txt/global/test.txt"
                pro=anamean(fpatch.replace("patch","global"),fpatch,groud,L)
                dic1['40X'].append(pro)
    for x in dic1:
#                     print(L[0],x,'mean-std',np.mean(np.array(dic1[x])),np.std(np.array(dic1[x])))
            #如果是五折一起投票的话，返回的就是概率值
        print (fivevote(dic1["40X"]))

In [ ]:
#compute various matrix on patient level
def anamean_patient(Global,Patch,groundtruth,Lxi):
    fileground=open(groundtruth,'r')
    Global=np.loadtxt(Global)    
    Patch=np.loadtxt(Patch)
    dic1={}
    dicpatient={}
    with open ("global/test_patient1.txt",'r')as f:
        lines=f.readlines()
        for line in lines:
            line=line.rstrip()
            dicpatient[line.split(" ")[0]]=line.split(" ")[1]
    for (num,picturename) in enumerate(fileground):        
        patientid=dicpatient[picturename.split("/")[-1].split(".")[0]]
#         patientid=picturename.split('/')[-2]
        reallabel=int(picturename[0:-1].split(' ')[-1])
        if patientid not in dic1:
            dic1[patientid]=[0,0,0,0]       
        
        patchvote=Patch[num*15:(num+1)*15]
        delline=[]
        
        ##计算投票
        dic1[patientid][reallabel+2]+=1
        for num2,x in enumerate(patchvote):
            if max(x)-min(x)<0.3:
                delline.append(num2)
        patchvote=np.delete(patchvote,delline,axis=0)
        siz,yy=patchvote.shape
        patchvote=np.sum(patchvote, axis=0)
        patchvote=patchvote/siz
        globalvote=Global[num:num+1]
        golovote=1.0
#         Lxi=[0,0,0,0]
        pre=Lxi[0]*patchvote+golovote*globalvote
        prelabel=pre.argmax()
        
        if prelabel==reallabel==1:
            dic1[patientid][1]+=1  
        elif prelabel==reallabel==0:
            dic1[patientid][0]+=1 
#计算只有patch,还要把dic1[patientid][0]+=1 变成0  dic1[patientid]=[1,0] 变[0,0]

#         for xx in patchvote:
#             prelabel=xx.argmax()
#             dic1[patientid][reallabel+2]+=1
#             if prelabel==reallabel==1:
#                 dic1[patientid][1]+=1 
#             elif prelabel==reallabel==0:
#                 dic1[patientid][0]+=1  
#计算acc
    PCscore=0.
    for  x in dic1:
        PCscore+=float((dic1[x][1]+dic1[x][0])/(dic1[x][2]+dic1[x][3]))
    Score=PCscore/len(dic1)
#计算F1
#     compute_label=1
#     Fre0=1.
#     for  x in dic1:
#         onefre=0.
#         if compute_label:       
#             if dic1[x][compute_label+2]!=0 and dic1[x][compute_label]!=0:
#                 precesion=dic1[x][1]/(dic1[x][1]+dic1[x][2]-dic1[x][0])
#                 recall=dic1[x][1]/(dic1[x][3])
#                 onefre=(2*precesion*recall)/(precesion+recall)
#         else:
#             if dic1[x][compute_label+2]!=0 and dic1[x][compute_label]!=0:
#                 precesion=dic1[x][0]/(dic1[x][0]+dic1[x][3]-dic1[x][1])
#                 recall=dic1[x][0]/(dic1[x][2])
#                 onefre=(2*precesion*recall)/(precesion+recall)
#         Fre0+=onefre
#     Score=Fre0/len(dic1)
                    
                    
#     res=open('result/result-patient-patch.txt','a+')
#     res.write(groundtruth.split('/')[-3]+" "+groundtruth.split('/')[-2]+" Score:"+str(Score)+'\n\n')
#     res.close()
    return Score
def fivevote_patient(list_pro):
    corr0=0
    corr1=0
    label0=0
    label1=0
    wrong=0    
    groud="global/test.txt"
    fileground=open(groud,'r')
    dicpatient={}
    with open ("global/test_patient1.txt",'r')as f:
        lines=f.readlines()
        for line in lines:
            line=line.rstrip()
            dicpatient[line.split(" ")[0]]=line.split(" ")[1]
    dicre={}
    
    for (num,picturename) in enumerate(fileground):
        patientid=dicpatient[picturename.split("/")[-1].split(".")[0]]
        reallabel=int(picturename[0:-1].split(' ')[-1])
        if patientid not in dicre:
            dicre[patientid]=[0,0,0,0]       
        dicre[patientid][reallabel+2]+=1
        ##使用sum的方式
#         pro_l=[0,0]
#         for ii in range(5):
#             pro_l+=list_pro[ii][num]
#         prelabel=pro_l[0].argmax()
        
        ##使用max的方式
        max_value=-1
        prelabel=-1
        for ii in range(5):
            for ii_2 in range(2):
                if list_pro[ii][num][0][ii_2]>max_value:
                    max_value=list_pro[ii][num][0][ii_2]
                    prelabel=ii_2    
        if prelabel==reallabel==0:
            dicre[patientid][0]+=1 
        elif prelabel==reallabel==1:
            dicre[patientid][1]+=1 
    PCscore=0.
    for  x in dicre:
        PCscore+=float((dicre[x][1]+dicre[x][0])/(dicre[x][2]+dicre[x][3]))
    Score=PCscore/len(dicre)
#     res=open('result/result-patient-11.txt','a+')
#     res.write(groud.split('/')[-1].split('_')[0]+" "+groud.split('/')[-2]+" correct0:"+str(corr0)+" correct1:"+str(corr1)+" label0:"+str(label0)+" label1:"+str(label1)+" pro:"+str(float(corr0+corr1)/float(corr0+corr1+wrong))+'\n\n')
#     res.close()
    return Score
if __name__=='__main__':
    if 'result-patient-11.txt' in os.listdir('result/'):
        os.remove('result/result-patient-11.txt')
    txtfile2="result/"
    filelist=search(txtfile2,'_patch')
    dic2={}
    dic2['40X']=[]
    Lxi=[6.2,5.9,8.5,5.4]
#     for xx in range(1):
#         for fpatch in filelist:
#                 gro=fpatch.split('/')[-1].split('.')[0]
#                 groud="/home/devin/Pictures/ICIAR2018_part2crop/tools/my_txt/global/test.txt"
#                 pro=anamean_patient(fpatch.replace("patch","global"),fpatch,groud,Lxi)
#                 dic2["40X"].append(float(pro))
#         for x in dic2:
#                 print(xx,x,'mean-std',np.mean(np.array(dic2[x])),np.std(np.array(dic2[x])))

#计算fivevote 的话，首先利用上面那个代码块的dic1,要再运行一下上面那个代码块
    for x in dic1:
#                     print(L[0],x,'mean-std',np.mean(np.array(dic1[x])),np.std(np.array(dic1[x])))
            #如果是五折一起投票的话，返回的就是概率值
        print fivevote_patient(dic1["40X"])